In [ ]:
!pip install PyMySQL
!pip install peewee

In [9]:
#  connection on EC2 RDS

import peewee
import pymysql

db = peewee.MySQLDatabase('myproject',
                          user='BB1032',
                          host='localhost',
                          password='IamBB1032',
                          port=5200)

In [ ]:
#  connection on localhost

import peewee
import pymysql

db = peewee.MySQLDatabase('myproject',
                          user='root',
                          host='localhost',
                          password='athena1225')

In [26]:
# connected successfully
# must bulid the schema before connected


db.get_columns('products')

[ColumnMetadata(name='id', data_type='int', null=False, primary_key=True, table='products'),
 ColumnMetadata(name='pro_id', data_type='int', null=False, primary_key=True, table='products'),
 ColumnMetadata(name='pro_name', data_type='longtext', null=True, primary_key=False, table='products'),
 ColumnMetadata(name='pro_brand', data_type='int', null=False, primary_key=False, table='products'),
 ColumnMetadata(name='pro_class', data_type='int', null=False, primary_key=False, table='products'),
 ColumnMetadata(name='pro_time', data_type='varchar', null=True, primary_key=False, table='products'),
 ColumnMetadata(name='pro_price', data_type='float', null=True, primary_key=False, table='products'),
 ColumnMetadata(name='pro_volumn', data_type='varchar', null=True, primary_key=False, table='products'),
 ColumnMetadata(name='pro_pic', data_type='longtext', null=True, primary_key=False, table='products'),
 ColumnMetadata(name='pro_score', data_type='float', null=True, primary_key=False, table='p

In [22]:
import datetime

class products(peewee.Model):
    pro_id = peewee.IntegerField()
    pro_name = peewee.TextField()
    pro_brand = peewee.IntegerField()
    pro_class = peewee.IntegerField()
    pro_time = peewee.CharField()
    pro_price = peewee.FloatField()
    #pro_volume = peewee.CharField()
    pro_pic = peewee.TextField()
    pro_score = peewee.FloatField()
    pro_color = peewee.TextField()
    RGB = peewee.CharField()
   
    
    class Meta:
        database = db

In [23]:
def unified_field(j_file):
    
    if 'listing_date' not in j_file.keys():
        j_file['listing_date']='null'
            
            #pprint(j_file)
    if 'product_color' not in j_file.keys():
        j_file['product_color']='one_color'
            
    if 'product_ID'not in j_file.keys():
        j_file['product_ID']='null'
            
    if 'product_name'not in j_file.keys():
        j_file['product_name']='null'
            
    if 'brand'not in j_file.keys():
        j_file['brand']='null'
            
    if 'price'not in j_file.keys():
        j_file['price']='null'
            
    if 'volumn'not in j_file.keys():
        j_file['volumn']='null'
    
    if 'score'not in j_file.keys():
        j_file['score']= 0

    
    if 'pro_picture'not in j_file.keys():
        j_file['pro_picture']='null'

    
    if 'pro_class'not in j_file.keys():
        j_file['pro_class']='%s'%reg_class
        
    if 'RGB' not in j_file.keys() :
        j_file['RGB'] = 'null'
        
        
    return j_file

In [13]:
def class_into_num(class_):
    
    if class_ == 'eyebrow' :
        class_ = '1'
    elif class_ == 'eye_liner':
        class_ = '2'
    elif class_ == 'base':
        class_ = '3'
    elif class_ == 'lip_stick':
        class_ = '4'
    else:
        print("out of classification")
        
    return class_

In [14]:
def insert_data(file_lists):
       
    dicts = ({'pro_id':file_lists[i]['product_ID'],
             'pro_name':file_lists[i]['product_name'],
             'pro_brand':file_lists[i]['brand'],
             'pro_class':'%s'%reg_class ,
             'pro_time':file_lists[i]['listing_date'],
             'pro_price':file_lists[i]['price'],
             'pro_pic':file_lists[i]['pro_picture'],
             'pro_score':file_lists[i]['score'],
             'pro_color':file_lists[i]['product_color'],
             'RGB':file_lists[i]['RGB'], 
            } for i in range(len(file_lists)))                
    with db.atomic():
        products.insert_many(dicts).execute()

In [15]:
def making_brand_numeric(uni_file) :
    
    for x in range(len(brands)) :
            
            if uni_file['brand'] not in brands[x].keys() :
                continue              
            else :
                br_num = brands.index({'%s'%uni_file['brand']:x})  
                values = brands[br_num]['%s'%uni_file['brand']]
                #print(values)
                uni_file['brand'] = values
                #print(uni_file['brand'])
                
    return uni_file

In [16]:
def unified_price(uni_file):
    if uni_file['price'] == 'null' :
        uni_file['price'] = 0.0
    else :
        try:
            price = uni_file['price'].split('NT$ ')[1]
            price = price/1
            #print(price)
                
        except :
            price = price.split('/')[0]
            #print(price)
             
        finally :
            uni_file['price'] = float(price)
            
    return uni_file        

In [17]:
def cleaned_score(uni_file):
    if uni_file['score'] == '-.-' :
        uni_file['score'] = 0.0
    else :
        uni_file['score'] = float(uni_file['score'])
        
    return uni_file

In [18]:
# Making a brand list

import glob
import json
import codecs
from pprint import pprint


dir_lists = ['eyebrow', 'base', 'eye_liner', 'lip_stick']

brand_lists = []

for dir_list in dir_lists:
    
    files_base = glob.glob('D:\\project\\urcosme\\%s\\*.json'%dir_list)
    
    for i in files_base :
        
        class_ = i.split('\\')[3]
        
        reg_class = class_into_num(class_)
        
          
    
        with codecs.open(i,'r','utf-8')as file:
            
            j_file = json.load(file)
            
            uni_file = unified_field(j_file)
            
            brand_lists.append(uni_file['brand'])
        
        
     
    
# distinct brand list 

unique_brand_lists = []

for brand in brand_lists:
     if brand not in unique_brand_lists :
            unique_brand_lists.append(brand)
            
unique_brand_lists
len(unique_brand_lists)


# give brand_id

brands = []

for i in unique_brand_lists:
    bb = {'%s'%i : unique_brand_lists.index(i)}
    #print(bb)
    brands.append(bb)
    
#brands




In [27]:
# inserting data ->eyebrow

import glob
import json
import codecs
from pprint import pprint

files_base = glob.glob('D:\\project\\urcosme\\eyebrow\\*.json')

file_path = 'D:\\project\\urcosme\\eyebrow\\*.json'

class_ = file_path.split('\\')[3]
#print(class_)
reg_class = class_into_num(class_)



file_lists = []

for i in files_base :
    
   
    class_ = i.split('\\')[3]
    #print(class_)
    reg_class = class_into_num(class_)
    #class_into_num(class_)
    
    with codecs.open(i,'r','utf-8')as file:
        
        j_file = json.load(file)
        
        uni_file = unified_field(j_file)
        #print(uni_file)?
        #print(uni_file['brand'])
        
        
        making_brand_numeric(uni_file)
        unified_price(uni_file) 
        cleaned_score(uni_file)
        
        
        
        file_lists.append(uni_file)
     
    
insert_data(file_lists)
        



    

        
        
       
        


In [28]:
# inserting data ->eye_liner

import glob
import json
import codecs
from pprint import pprint

files_base = glob.glob('D:\\project\\urcosme\\eye_liner\\*.json')

file_path = 'D:\\project\\urcosme\\eye_liner\\*.json'

class_ = file_path.split('\\')[3]
#print(class_)
reg_class = class_into_num(class_)



file_lists = []

for i in files_base :
    
   
    class_ = i.split('\\')[3]
    #print(class_)
    reg_class = class_into_num(class_)
    #class_into_num(class_)
    
    with codecs.open(i,'r','utf-8')as file:
        
        j_file = json.load(file)
        
        uni_file = unified_field(j_file)
        #print(uni_file)?
        #print(uni_file['brand'])
        
        
        making_brand_numeric(uni_file)
        unified_price(uni_file) 
        cleaned_score(uni_file)
        
        
        
        file_lists.append(uni_file)
     
    
insert_data(file_lists)
        


In [29]:
# inserting data ->base

import glob
import json
import codecs
from pprint import pprint

files_base = glob.glob('D:\\project\\urcosme\\base\\*.json')

file_path = 'D:\\project\\urcosme\\base\\*.json'

class_ = file_path.split('\\')[3]
#print(class_)
reg_class = class_into_num(class_)



file_lists = []

for i in files_base :
    
   
    class_ = i.split('\\')[3]
    #print(class_)
    reg_class = class_into_num(class_)
    #class_into_num(class_)
    
    with codecs.open(i,'r','utf-8')as file:
        
        j_file = json.load(file)
        
        uni_file = unified_field(j_file)
        #print(uni_file)?
        #print(uni_file['brand'])
        
        
        making_brand_numeric(uni_file)
        unified_price(uni_file) 
        cleaned_score(uni_file)
        
        
        
        file_lists.append(uni_file)
     
    
insert_data(file_lists)
        

In [30]:
# inserting data ->lip_stick

import glob
import json
import codecs
from pprint import pprint

files_base = glob.glob('D:\\project\\urcosme\\lip_stick\\*.json')

file_path = 'D:\\project\\urcosme\\lip_stick\\*.json'

class_ = file_path.split('\\')[3]
#print(class_)
reg_class = class_into_num(class_)



file_lists = []

for i in files_base :
    
   
    class_ = i.split('\\')[3]
    #print(class_)
    reg_class = class_into_num(class_)
    #class_into_num(class_)
    
    with codecs.open(i,'r','utf-8')as file:
        
        j_file = json.load(file)
        
        uni_file = unified_field(j_file)
        #print(uni_file)?
        #print(uni_file['brand'])
        
        
        making_brand_numeric(uni_file)
        unified_price(uni_file) 
        cleaned_score(uni_file)
        
        
        
        file_lists.append(uni_file)
     
    
insert_data(file_lists)
        